In [ ]:
import os 
import sys
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import numpy as np

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [ ]:
#Importing Configs
# Define the path where config.py is located
#Mac
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
#Linux
#os.chdir('/home/kwnabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models
finbert_models = config.finbert_models
database = config.Local_Database

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
# Model_Folder = config.Sentiment_models
# Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2


In [ ]:
df1 = pd.read_csv(f"{finbert_models}/{Body}/{Model}/{Body}_{Model}_finbert_model_short.csv")
df2 = pd.read_csv(f"{finbert_models}/{Body}/{Model}/{Body}_{Model}_finbert_model_short_2.csv")

df1 = df1[['date', 'sentiment']]

df1 = df1.groupby(['date']).mean().reset_index()

In [ ]:
Fed_list = ["Statements","Minutes"]


df1 = pd.DataFrame()
for i in Fed_list:
    Body = "Fed"
    Model = i
    Model_Subfolder = f'/{Body} Texts/{Model}'
    Model_Folder = config.Sentiment_models
    dft = pd.read_csv(f"{finbert_models}/{Body}/{Model}/{Body}_{Model}_finbert_model_short.csv")
    
    dft = dft[['date', 'sentiment']]
    df1 = pd.concat([df1, dft])
    

df1 = df1.groupby(['date']).mean().reset_index()

In [ ]:
ffr = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/FFRF/FFRF.csv')
#Convert Date to datetime period days 
ffr['Date'] = pd.to_datetime(ffr['Date'])
ffr = ffr[['Date', 'Price', 'Vol.']]
#Rename Pirce to ffr_price and Vol. to ffr_vol
ffr = ffr.rename(columns={'Price': 'ffr_price', 'Vol.': 'ffr_vol'})
#on ffr_vol remove the k at the end of the number and convert to float
ffr['ffr_vol'] = ffr['ffr_vol'].str.replace('K', '')
ffr['ffr_vol'] = ffr['ffr_vol'].astype(float)
ffr['ffr_vol'] = ffr['ffr_vol'] * 1000

print(ffr.head())

tb10y = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/10Y Treasury/T10YFF.csv')

tb10y['DATE'] = pd.to_datetime(tb10y['DATE'])
tb10y = tb10y.rename(columns={'T10YFF': 'tb10y'})

tb10y = tb10y[tb10y['tb10y'] != '.']
print(tb10y.head())

ir1 = pd.read_csv(f"{database}/Other Data/2YIR.csv")
ir1 = ir1.rename(columns={'DATE': 'date', 'DGS2': 'IR2'})
ir1['date'] = pd.to_datetime(ir1['date'])
ir1 = ir1[ir1['IR2'] != '.']

In [ ]:
#Convert date to datetime
df1['date'] = pd.to_datetime(df1['date'])
df1 = df1.merge(ffr, how='left', left_on='date', right_on='Date')
df1 = df1.drop(columns=['Date'])

df1 = df1.merge(tb10y, how='left', left_on='date', right_on='DATE')
df1 = df1.drop(columns=['DATE'])

df1 = df1.merge(ir1, how='left', left_on='date', right_on='date')

#Convert tb10y and IR2 to float
df1['tb10y'] = df1['tb10y'].astype(float)
df1['IR2'] = df1['IR2'].astype(float)

print(df1.dtypes)
#df1 = df1.dropna()
print(df1.head(-10))   


In [ ]:
df_temp = pd.read_csv(f"/Users/kylenabors/Documents/Database/Other Data/SPY/SPYs.csv")

In [ ]:
spy = df_temp.copy(deep=True)
spy = spy[['DATE', 'CRSPTM1', 'TIME']]
spy = spy.rename(columns={'DATE': 'date', 'CRSPTM1': 'spy_price', 'TIME': 'time'})
spy['spy_price'] = spy['spy_price']
spy['date2'] = spy['date'].copy(deep=True)
spy["date"] = spy["date"] + " " + spy["time"]
spy["date"] = pd.to_datetime(spy["date"])
spy["date2"] = pd.to_datetime(spy["date2"])
print(spy.head()) 
print(spy.dtypes)

In [ ]:
spy = spy.merge(df1, how='left', left_on='date2', right_on='date')
spy = spy.drop(columns=['date_y'])
spy = spy.rename(columns={'date_x': 'date'})
spy = spy[['date', 'spy_price', 'sentiment', 'time']]
spy = spy.dropna()


spy["positive"] = np.where(spy["sentiment"] > 0.01, 1, 0)
spy["negative"] = np.where(spy["sentiment"] < -0.01, 1, 0)
spy["neutral"] = np.where((spy["sentiment"] <= 0.01) & (spy["sentiment"] >= -0.01), 1, 0)
print(spy.head())

spy_pos = spy[spy['positive'] == 1]
spy_neg = spy[spy['negative'] == 1]
spy_neu = spy[spy['neutral'] == 1]

In [ ]:
#Caculate mean of sentiment overall 
spy_mean = spy['sentiment'].mean()
spy_mean = round(spy_mean, 4)
print(spy_mean)

In [ ]:
spy = spy[['date', 'spy_price',]]

# Set the 'date_time' column as the index
spy.set_index('date', inplace=True)

# Shift the 'price' column by 5 minutes
spy['price_5_min_ago'] = spy['spy_price'].shift(freq='5T')

# Calculate the difference
#spy['diff'] = spy['spy_price'] - spy['price_5_min_ago']
spy['diff'] = 100 * np.log10(spy['spy_price'] / spy['price_5_min_ago'])

#For diff remove observations that are below the 5th percentile or above the 95th percentile
spy = spy[spy['diff'] >= spy['diff'].quantile(.05)]
spy = spy[spy['diff'] <= spy['diff'].quantile(.95)]

# Reset the index if you want 'date_time' back as a column
spy.reset_index(inplace=True)

spy = spy[['date', 'spy_price', 'diff']]
spy['time'] = spy['date'].dt.time
spy_2 = spy.groupby(['time']).agg(["mean", "var"]).reset_index()
spy_2.columns = ['_'.join(col).strip() for col in spy_2.columns.values]
spy_2['spy_price_sd'] = spy_2['spy_price_var'] ** 0.5
spy_2['diff_sd'] = spy_2['diff_var'] ** 0.5
spy_2 = spy_2[['time_', 'spy_price_mean', 'spy_price_sd', 'diff_mean', 'diff_sd']]
spy_2 = spy_2.rename(columns={'time_': 'time'})
spy_2 = spy_2.dropna()
print(spy_2.head())

spy_2['time'] = pd.to_datetime(spy_2['time'], format='%H:%M:%S')
print(spy_2.dtypes) 
spy_2 = spy_2.sort_values(by=['time'])
spy_2 = spy_2.set_index('time')
spy_2 = spy_2.between_time('12:00:00', '16:00:00')
spy_2 = spy_2.reset_index()
#Remove the date from the time column
spy_2['time'] = spy_2['time'].dt.time
print(spy_2.head())


In [ ]:

spy_pos = spy_pos[['date', 'spy_price',]]

# Set the 'date_time' column as the index
spy_pos.set_index('date', inplace=True)

# Shift the 'price' column by 5 minutes
spy_pos['price_5_min_ago'] = spy_pos['spy_price'].shift(freq='5T')

# Calculate the difference
spy_pos['diff'] = 100 * np.log10(spy_pos['spy_price'] / spy_pos['price_5_min_ago'])

#For diff remove observations that are below the 5th percentile or above the 95th percentile
spy_pos = spy_pos[spy_pos['diff'] >= spy_pos['diff'].quantile(.05)]
spy_pos = spy_pos[spy_pos['diff'] <= spy_pos['diff'].quantile(.95)]

# Reset the index if you want 'date_time' back as a column
spy_pos.reset_index(inplace=True)

spy_pos = spy_pos[['date', 'spy_price', 'diff']]
spy_pos['time'] = spy_pos['date'].dt.time
spy_pos_2 = spy_pos.groupby(['time']).agg(["mean", "var"]).reset_index()
spy_pos_2.columns = ['_'.join(col).strip() for col in spy_pos_2.columns.values]
spy_pos_2['spy_price_sd'] = spy_pos_2['spy_price_var'] ** 0.5
spy_pos_2['diff_sd'] = spy_pos_2['diff_var'] ** 0.5
spy_pos_2 = spy_pos_2[['time_', 'spy_price_mean', 'spy_price_sd', 'diff_mean', 'diff_sd']]
spy_pos_2 = spy_pos_2.rename(columns={'time_': 'time'})
spy_pos_2 = spy_pos_2.dropna()
print(spy_pos_2.head())

spy_pos_2['time'] = pd.to_datetime(spy_pos_2['time'], format='%H:%M:%S')
print(spy_pos_2.dtypes) 
spy_pos_2 = spy_pos_2.sort_values(by=['time'])
spy_pos_2 = spy_pos_2.set_index('time')
spy_pos_2 = spy_pos_2.between_time('12:00:00', '16:00:00')
spy_pos_2 = spy_pos_2.reset_index()
spy_pos_2['time'] = spy_pos_2['time'].dt.time
print(spy_pos_2.head())

In [ ]:

spy_neg = spy_neg[['date', 'spy_price',]]

# Set the 'date_time' column as the index
spy_neg.set_index('date', inplace=True)

# Shift the 'price' column by 5 minutes
spy_neg['price_5_min_ago'] = spy_neg['spy_price'].shift(freq='5T')

# Calculate the difference
spy_neg['diff'] = 100 * np.log10(spy_neg['spy_price'] / spy_neg['price_5_min_ago'])

#For diff remove observations that are below the 5th percentile or above the 95th percentile
spy_neg = spy_neg[spy_neg['diff'] >= spy_neg['diff'].quantile(.05)]
spy_neg = spy_neg[spy_neg['diff'] <= spy_neg['diff'].quantile(.95)]

# Reset the index if you want 'date_time' back as a column
spy_neg.reset_index(inplace=True)

spy_neg = spy_neg[['date', 'spy_price', 'diff']]
spy_neg['time'] = spy_neg['date'].dt.time
spy_neg_2 = spy_neg.groupby(['time']).agg(["mean", "var"]).reset_index()
spy_neg_2.columns = ['_'.join(col).strip() for col in spy_neg_2.columns.values]
spy_neg_2['spy_price_sd'] = spy_neg_2['spy_price_var'] ** 0.5
spy_neg_2['diff_sd'] = spy_neg_2['diff_var'] ** 0.5
spy_neg_2 = spy_neg_2[['time_', 'spy_price_mean', 'spy_price_sd', 'diff_mean', 'diff_sd']]
spy_neg_2 = spy_neg_2.rename(columns={'time_': 'time'})
spy_neg_2 = spy_neg_2.dropna()
print(spy_neg_2.head())

spy_neg_2['time'] = pd.to_datetime(spy_neg_2['time'], format='%H:%M:%S')
print(spy_neg_2.dtypes) 
spy_neg_2 = spy_neg_2.sort_values(by=['time'])
spy_neg_2 = spy_neg_2.set_index('time')
spy_neg_2 = spy_neg_2.between_time('12:00:00', '16:00:00')
spy_neg_2 = spy_neg_2.reset_index()
spy_neg_2['time'] = spy_neg_2['time'].dt.time
print(spy_neg_2.head())

In [ ]:

spy_neu = spy_neu[['date', 'spy_price',]]

# Set the 'date_time' column as the index
spy_neu.set_index('date', inplace=True)

# Shift the 'price' column by 5 minutes
spy_neu['price_5_min_ago'] = spy_neu['spy_price'].shift(freq='5T')

# Calculate the difference
spy_neu['diff'] = 100 * np.log10(spy_neu['spy_price'] / spy_neu['price_5_min_ago'])

#For diff remove observations that are below the 5th percentile or above the 95th percentile
spy_neu = spy_neu[spy_neu['diff'] > spy_neu['diff'].quantile(.05)]
spy_neu = spy_neu[spy_neu['diff'] < spy_neu['diff'].quantile(.95)]

# Reset the index if you want 'date_time' back as a column
spy_neu.reset_index(inplace=True)

spy_neu = spy_neu[['date', 'spy_price', 'diff']]
spy_neu['time'] = spy_neu['date'].dt.time
spy_neu_2 = spy_neu.groupby(['time']).agg(["mean", "var"]).reset_index()
spy_neu_2.columns = ['_'.join(col).strip() for col in spy_neu_2.columns.values]
spy_neu_2['spy_price_sd'] = spy_neu_2['spy_price_var'] ** 0.5
spy_neu_2['diff_sd'] = spy_neu_2['diff_var'] ** 0.5
spy_neu_2 = spy_neu_2[['time_', 'spy_price_mean', 'spy_price_sd', 'diff_mean', 'diff_sd']]
spy_neu_2 = spy_neu_2.rename(columns={'time_': 'time'})
spy_neu_2 = spy_neu_2.dropna()
print(spy_neu_2.head())

spy_neu_2['time'] = pd.to_datetime(spy_neu_2['time'], format='%H:%M:%S')
print(spy_neu_2.dtypes) 
spy_neu_2 = spy_neu_2.sort_values(by=['time'])
spy_neu_2 = spy_neu_2.set_index('time')
spy_neu_2 = spy_neu_2.between_time('12:00:00', '16:00:00')
spy_neu_2 = spy_neu_2.reset_index()
spy_neu_2['time'] = spy_neu_2['time'].dt.time
print(spy_neu_2.head())

In [ ]:
import datetime

# Convert datetime.time to datetime.datetime
def time_to_datetime(time_obj):
    return datetime.datetime.combine(datetime.date.today(), time_obj)

spy_2['time'] = spy_2['time'].apply(time_to_datetime)
spy_pos_2['time'] = spy_pos_2['time'].apply(time_to_datetime)
spy_neg_2['time'] = spy_neg_2['time'].apply(time_to_datetime)
spy_neu_2['time'] = spy_neu_2['time'].apply(time_to_datetime)

In [ ]:
# Create a list of x-axis tick positions
num_ticks = 30  # Adjust this number as needed
x_tick_positions_0 = spy_2['time'].iloc[::len(spy_2) // num_ticks].tolist()

fig, axs = plt.subplots(2, 1, figsize=(30, 30))
#axs = [axs, axs.twiny()]

import matplotlib.dates as mdates

# Now you can plot
axs[0].plot(spy_2['time'], spy_2['diff_sd'], color='blue', linewidth=1)
axs[1].plot(spy_pos_2['time'], spy_pos_2['diff_sd'], color='green', linewidth=0.5)
axs[1].plot(spy_neg_2['time'], spy_neg_2['diff_sd'], color='red', linewidth=0.5)
#axs[1].plot(spy_neu_2['time'], spy_neu_2['diff_sd'], color='orange', linewidth=0.5)

# Format the x-axis to display only the time
axs[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
axs[1].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

# Set Y axis scales so that they both fit evenly on the chart
axs[0].set_ylim(0.005, 0.055)
axs[1].set_ylim(0.005, 0.055)

axs[0].tick_params(axis='x', rotation=45, labelsize=15)
axs[1].tick_params(axis='x', rotation=45, labelsize=15)
#Set y label
axs[0].set_ylabel('Standard Deviation', fontsize=30)
axs[1].set_ylabel('Standard Deviation', fontsize=30)

axs[0].set_title(f'SPY (S&P 500)', fontsize=50)

plt.show()